# Welcome to the PyLibSIMBA walkthrough


We will go through the simple steps it takes to submit a message to a blockchain, and verify the message arrived.

## First lets import PyLibSIMBA

This is already installed in the notebook so we can do

In [1]:
import pylibsimba


## Test 1 - Generate a Wallet

This holds the public and private keys used to sign a transaction, and creates an address to send the transactions from.

We will import a helper class and generate a wallet.

### Advanced usage:

If you create a contract on simbachain.com which uses a blockchain which requires currency, ensure that a wallet with
 that address has the correct balance (via their website, spigot etc), and initialise the Wallet object above with 
 the mnemonic associated with that wallet. The Wallet object will then have the same address and can be used to sign 
 transactions on that blockchain.

In [2]:
from pylibsimba.wallet import Wallet


Now we can generate a wallet to use on the SIMBAChain test blockchain.

In [3]:
wallet = Wallet(None)
wallet.generate_wallet('test1234')
addr = wallet.get_address()
print(addr)


0xc341e8e093f44d9149e20f39317da93def0dd02b


## Test 2 - Get the SIMBAChain object. 

For the this step we need to import a few things.

Specifically we need the ability to get a SIMBAChain object.

Most of the outputs are in JSON format, so we import that too.

In [4]:
from pylibsimba import get_simba_instance
import json


Now we can get a SIMBAChain instance, which is used for all interactions with the SIMBAChain API.

This requires a few parameters:

* url : The API URL
	* We'll use the test endpoint from simbachain.com
* wallet : The Wallet to use
	* This is the wallet we just made above
* api_key : (Optional) The API key
	* This is a test API pre generated for this example.
	* Please register at simbachain.com to gain the ability to create your own contracts and access them via you own 
	API key.
* management_key : (Optional) The Management API key
	* As for the api_key, but unused for now.
        

In [5]:
simba = get_simba_instance(
    'https://api.simbachain.com/v1/libSimba-SimbaChat-Quorum/',
    wallet,
    '04d1729f7144873851a745d2ae85639f55c8e3de5aea626a2bcd0055c01ba6fc',
    '')


### Using the SIMBAChain instance

Lets check the balance of our wallet. To do this we simply get_balance() from the simba instance we've just created.

The output is a dict, so we'll use json to dump it to a string.

In [6]:
balance = simba.get_balance()
print("Balance: {}".format(json.dumps(balance, indent=4)))


Balance: {
    "amount": -1,
    "currency": "",
    "poa": true
}


The output shows the balance as -1, but we can also see that ```"poa": true``` meaning this is a Proof of Authority 
blockchain, so no currency is required.


## Test 3 - Calling a method on a Smart Contract via SIMBAChain.com

First, we need the parameters to call the method with.

As it's an example method, the params are given below. (We'll look at how to get the paramters for a different method later)

In [7]:
method_params = {
    'assetId': "0xbad65ff688a28efdd17d979c12f0ab2e2de305dbc8a2aa6be45ed644da822cfb",
    'name': "A Test Room",
    'createdBy': "PyLibSIMBA",
}


Call the method with the parameters. That's it!


In [8]:
resp = simba.call_method('createRoom', method_params)
print("Successful submitting? {}".format(resp.transaction_id))


Successful submitting? 76876d0d-888d-46ae-b362-daa9facef855


### Did it work?

There is a delay between submitting a transaction, and it appearing on the blockchain.

We have the option to "send and forget", or check to see if it has "deployed" successfully. 

To be completely sure, we can run wait_for_success_or_error() to check. 

Be aware this can take some time, and a variety of errors can be thrown if a problem is detected. Please check the documentation on Exceptions to learn about the types of errors caught by the PyLibSIMBA SDK.

In [9]:
try:
    final_resp = simba.wait_for_success_or_error(resp.transaction_id)
    print("Successful? {}".format(final_resp))

except Exception as e1:
    print("Failure! {}".format(e1))


Successful? True


## Test 4 - Calling a method and submitting files

Similar to above, we need some method parameters, and we'll add some files too.

In [10]:
method_params = {
    'assetId': "A Test Room",
    'chatRoom': "A Test Room",
    'message': "Hello World",
    'sentBy': "PyLibSIMBA"
}

files =  ["../tests/test_files/test file 1.txt", "../tests/test_files/test file 2.txt"]


Instead of call_method() we use call_method_with_file()

To be sure it worked, we'll add a wait again. This operation will take several seconds to complete.

N.B, Ensure the files exist in the given location, or the method will return an error.

In [11]:
try:
    resp = simba.call_method_with_file('sendMessage', method_params, files)
    print("Successful submitting? {}".format(resp.transaction_id))

    resp = simba.wait_for_success_or_error(resp.transaction_id)
    print("Successfully deployed? {}".format(resp))
except Exception as e1:
    print("Something went wrong: {}".format(e1))


Successful submitting? bae59c2b-5838-4757-b3bf-a88cc831c314
Successfully deployed? True


In [12]:
!ls ../tests


example.sol  file_0.txt  __init__.py  test_files
examples.py  File1.txt	 __pycache__  the_bundle.zip


## Test 5 - A list of transactions for the method "createRoom"

We can get a list of all transactions for the method "createRoom". 

The result is a PagedResponse class, which holds information about the number of results and a way to 'page' through them.

We can also filter so we only see the transactions that we have created, using createdBy_exact.

The output from this can be extensive and very detailed, so we'll use json.dumps() to "pretty print" it.

In [13]:
method_params = {
    'createdBy_exact': "PyLibSIMBA"
}
result_pages = simba.get_method_transactions('createRoom', method_params)

print("Number of results for transaction {}: {}".format('createRoom', result_pages.count()))

print("Got data : \n{}".format(
    json.dumps(
        result_pages.data(), indent=4
    )
))


Number of results for transaction createRoom: 52
Got data : 
[
    {
        "id": "76876d0d888d46aeb362daa9facef855",
        "parent_id": "0xbad65ff688a28efdd17d979c12f0ab2e2de305dbc8a2aa6be45ed644da822cfb",
        "method": "1a8525b3b93a449290c45d38c2a024fa",
        "payload": {
            "raw": {
                "to": "0x6f84419344a1caa3c87ecfddc5cdb5272eb24be4",
                "gas": "0x72df",
                "data": "0x1ea7c445000000000000000000000000000000000000000000000000000000000000006000000000000000000000000000000000000000000000000000000000000000e000000000000000000000000000000000000000000000000000000000000001200000000000000000000000000000000000000000000000000000000000000042307862616436356666363838613238656664643137643937396331326630616232653264653330356462633861326161366265343565643634346461383232636662000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000b41205465737420526f6f6d000000000000000000000000

## Test 6 - Get an existing example transaction object by the transaction ID.

Use the example transaction ID given, or one from previous called methods above.

In [14]:
transaction_id = "97b56a4dd3ff4fe7820f46a7101f72e2"
txn = simba.get_transaction(transaction_id)

print("Transaction : \n{}".format(
    json.dumps(txn, indent=4)
))


Transaction : 
{
    "id": "97b56a4dd3ff4fe7820f46a7101f72e2",
    "parent_id": "\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0 \\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\u000bA Test Room\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0",
    "method": "8082a89449cc42a4ae65e58aeb90569c",
    "payload": {
        "raw": {
            "to": "0x6f84419344a1caa3c87ecfddc5cdb5272eb24be4",
            "gas": "0x175f7",
            "data": "0x53df945100000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000000001a000000000000000000000000000000000000000000000000000000000000002a000000000000000000000000000000000000000000000000000000000000003a000000000000000000000000000000000000000000000000000000000000004a000000000000000000000000000000000000000000000000000000000000000c230783030303030303030303030303030303030303030303030303030

## Test 7 - Get the Transaction Metadata object for an existing example, by the transaction hash.


In [15]:
transaction_hash = "0x7565461be84259d5e365c2c3225696a6d74245f1eca1ecc050b1fedd5a4a1f4d"
txn_metadata = simba.get_bundle_metadata_for_transaction(transaction_hash)
print("Transaction Metadata: \n{}".format(json.dumps(txn_metadata, indent=4)))


Transaction Metadata: 
{
    "manifest": [
        {
            "name": "File1.txt",
            "mimetype": "text/plain",
            "size": 12
        },
        {
            "name": "File2.txt",
            "mimetype": "text/plain",
            "size": 10
        }
    ],
    "meta": {}
}



## Test 8 - Get a bundle of files from a given transaction, from the transaction hash
Writes the bundle to "the_bundle.zip".

This implementation sets stream=True so that the requests module doesn't download the whole bundle into memory
first.

We can also check for errors with raise_for_status()


In [16]:
transaction_hash = "0x7565461be84259d5e365c2c3225696a6d74245f1eca1ecc050b1fedd5a4a1f4d"
req = simba.get_bundle_for_transaction(transaction_hash, stream=True)
req.raise_for_status()


### Writing the "bundle" file to disk

The bundle of files is actually a zip file with all of the files submitted in the transaction, along with some metadata used to check their validity.

Writing the file is done as usual with the *python requests* package.

To check the file was written to disk, we do a isfile() check


In [17]:
output_file = 'the_bundle.zip'
with open('the_bundle.zip', 'wb') as f:
    for chunk in req.iter_content(chunk_size=8192):
        if chunk:  # filter out keep-alive new chunks
            f.write(chunk)

import os      
print("Wrote file {}: {}".format(
    output_file,
    os.path.isfile(os.path.abspath(output_file)))
)


Wrote file the_bundle.zip: True



## Test 9 - Get the first file from a bundle for a given transaction, from the transaction hash

Writes the file to "file_0.txt"

Very similar to the example above, but specifying an index from the list of files submitted in the transaction.


In [18]:
transaction_hash = "0x7565461be84259d5e365c2c3225696a6d74245f1eca1ecc050b1fedd5a4a1f4d"

req = simba.get_file_from_bundle_for_transaction(transaction_hash, 0, stream=True)
req.raise_for_status()

output_file = 'file_0.txt'
with open(output_file, 'wb') as f:
    for chunk in req.iter_content(chunk_size=8192):
        if chunk:  # filter out keep-alive new chunks
            f.write(chunk)
            
print("Wrote file {}: {}".format(
    output_file,
    os.path.isfile(os.path.abspath(output_file)))
)


Wrote file file_0.txt: True


## Test 10 - Get a file by name, from a bundle for a given transaction, from the transaction hash

Again, as above, but getting the file by name.

Writes the file to "File1.txt"


In [19]:
transaction_hash = "0x7565461be84259d5e365c2c3225696a6d74245f1eca1ecc050b1fedd5a4a1f4d"

filename = "File1.txt"
req = simba.get_file_from_bundle_by_name_for_transaction(transaction_hash, filename, stream=True)
req.raise_for_status()

output_file = 'File1.txt'
with open(output_file, 'wb') as f:
    for chunk in req.iter_content(chunk_size=8192):
        if chunk:  # filter out keep-alive new chunks
            f.write(chunk)
            
print("Wrote file \n{}: {}".format(
    output_file,
    os.path.isfile(os.path.abspath(output_file)))
)


Wrote file 
File1.txt: True


## Test 12 - Get the organisations this user belongs to

This is useful when performing more low-level API calls.


In [20]:
paged_response = simba.get_organisations()

for org in paged_response.data():
    print(org['id'])


b8f9e9a65aff4814abf0abaa30213c1f
390c95be07d84d4bbff3579372e4a732
b6f2ff06f3824d0699bf14ed3edfa8c9
7e4afdaa4f8d49118a0655c7ad8fc1dd
773c73a1c2cf435f989fa062f69ea0cd
5cd5cef4cabb4b009e00b6b3ff45ee08
d7efc15bbafc4bf1827acb5bb3d3d8c5


## Test 13 - Push arbitrary solidity code to simbachain.com

The create_contract() method takes four parameters:

* The simba object
* The path to a file containing the solidity code
* The name to call the Smart Contract. This will be shown in the dashboard.
* An organisation id. A user can be a member of multiple organisations, so this is the organisation the Smart 
Contract will be associated with.

A contract name must be unique within an organisation, so we will add a timestamp to the name so this test
organisation will accept it.
    

In [21]:
from _datetime import datetime
from pylibsimba import util

response = util.create_contract(
    simba,
    '../tests/example.sol',
    'example_contract_{}'.format(datetime.now().isoformat()),
    '5cd5cef4cabb4b009e00b6b3ff45ee08'
)
print("Wrote contract : \n{}".format(
    json.dumps(response.json(), indent=4)
))


Wrote contract : 
{
    "id": "4706e8811c904b8b8ec06b179029abbc",
    "name": "example_contract_2020-03-20T16:30:27.067898",
    "version": "2020-03-20T16:30:32.829557+00:00",
    "created_on": "2020-03-20T16:30:32.829557+00:00",
    "edited_on": "2020-03-20T16:30:32.829557+00:00",
    "err": null,
    "organisation_id": "5cd5cef4-cabb-4b00-9e00-b6b3ff45ee08",
    "is_public": false,
    "save_message": "No Save Message",
    "type": "solidity",
    "mode": "code",
    "organisation_name": "test",
    "metadata": "eyJjb250cmFjdCI6IHsibmFtZSI6ICJBcHBsaWNhdGlvbiIsICJhc3NldHMiOiB7fSwgInNvdXJjZSI6IHsibGFuZyI6ICJzb2xpZGl0eSIsICJ2ZXJzaW9uIjogIl4wLjQuMjQifSwgIm1ldGhvZHMiOiB7ImNvbXBsZXhFdmVudCI6IHsicGFyYW1zIjogW3sibmFtZSI6ICJhc3NldElkIiwgInR5cGUiOiAic3RyaW5nIn0sIHsibmFtZSI6ICJhU3RyaW5nIiwgInR5cGUiOiAic3RyaW5nIn0sIHsibmFtZSI6ICJzb21lQnl0ZXMiLCAidHlwZSI6ICJieXRlczMyIn0sIHsibmFtZSI6ICJhQnl0ZSIsICJ0eXBlIjogImJ5dGVzMSJ9LCB7Im5hbWUiOiAiYVVpbnQiLCAidHlwZSI6ICJ1aW50MjU2In0sIHsibmFtZSI6ICJhbkFkZHJlc3Mi

## Get in touch!

If you have any issues with the demo above, please let us know via the GitHub issues pages, <https://github.com/SIMBAChain/PyLibSIMBA/issues>